In [1]:
from collections import namedtuple
from IPython.display import display, Latex, Math, HTML, Markdown
#from pylab import *
import ipywidgets as widgets
import bqplot.pyplot as plt
from bqplot.interacts import BrushSelector
from bqplot import Tooltip
from ipyfilechooser import FileChooser

In [2]:
# set up definitions
fieldnames = """vibration obs diff predicted intensity lowerE upperE J_upper K_upper, L2_upper L_upper
J_lower K_lower L2_lower L_lower unc"""
dataline = namedtuple("dataline", fieldnames)

In [3]:
fc = FileChooser('.')
fc.filter_patern = ['*.txt']
fc.title = '<b>Select Symrot Output File</b>'

display(fc)

FileChooser(path='C:\Users\rmcrae\OneDrive - Eastern Washington University\Rotation 2022\binder', filename='',…

In [4]:
strL = "<font color='blue'>L' = {}</font>"
strK = "<font color='magenta'>K' = {}</font>"
strL2 = "<font color='green'>L2' = {}</font>"
strVib = "{}"
lblLayout = widgets.Layout(display = 'flex', justify_content = "center", width = "50px")
vibLayout = widgets.Layout(display = 'flex', justify_content = "center")
tipLayout = widgets.Layout(display = 'flex', align_items = 'center', border = "solid green 1px", width = "250px",
                          margin = "0px 0px 0px 0px")
#def MakeLabel(item):
#    lblL = widgets.HTML(value = strL.format(item.L_upper), layout = lblLayout)
#    lblK = widgets.HTML(value = strK.format(item.K_upper), layout = lblLayout)
#    lblL2 = widgets.HTML(value = strL2.format(item.L2_upper), layout = lblLayout)
#    lblVib = widgets.Label(value = item.vibration, layout = vibLayout)
#    newlayout = widgets.Layout(display = 'flex', justify_content = 'center', width='200px')
#    row2 = widgets.HBox([lblL, lblK, lblL2], layout = newlayout)
#    return widgets.VBox([lblVib, row2], layout = tipLayout)

# new attempt:  just write one monolithic HTML to Output
strLabel = '''<b style='text-align:center'>{}</b><br>
<font color='blue'>L' = {}</font> &emsp;<font color='magenta'>K' = {}</font> &emsp;<font color='green'>L2' = {}</font>
              '''
txtLabel = '''{}
L' = {}   K'' = {}   L2' = {}'''
widgLayout = widgets.Layout(width = "150px")
def MakeLabel(item):
    lblTest = widgets.HTML(value = strLabel.format(item.vibration, item.L_upper, item.K_upper, item.L_upper), layout = widgLayout)
    lblText = strLabel.format(item.vibration, item.L_upper, item.K_upper, item.L_upper)
    return lblText



In [25]:
widget_container = widgets.Output()
testing_output = widgets.Output()

class SymRotViewer:
    def __init__(self, data=None):
        # retain data internally
        self.data = data
        
        # set figure parameters
        fig_layout = widgets.Layout(height = "500px", width = "750px")
        self.fig = plt.figure("Vibrational Energies", layout = fig_layout)
        plt.clear()
        self.markerShapes = ['square', 'circle', 'diamond', 'cross', 'triangle-up', 'triangle-down']
        self.markerColors = ['blue','green','cyan', 'magenta', 'gold', 'black', 'orange']
        self.plots = {}
        
        
        # set keys and set default vibration to first key:
        self.keys = [] if not data else list(data.keys())
        #vibration = self.keys[0]
        #print("Created data set with keys ", self.keys, vibration)
        
        # initialize output for ToolTips
        outLayout = widgets.Layout(display = 'flex', justify_content = 'flex-start', border = "solid blue 1px",
                                                        # width = "300px", height = "120px", 
                                                          align_content = "flex-start",
                                                          overflow = "hidden",
                                                          align_itmes = "flex-start",)
        self.out = widgets.Output(layout = outLayout)
        self.err_out = widgets.Output(layout = widgets.Layout(border = "solid red 1px"))
        #self.out = widgets.Output()
        self.strLabel = '''<b style='text-align:center'>{}</b><br>
        <font color='blue'>L' = {}</font> &emsp;
        <font color='magenta'>K' = {}</font> &emsp;<font color='green'>L2' = {}</font>
              '''
        widgLayout = widgets.Layout(width = "150px")
        self.html_out = widgets.HTML(value = strLabel, layout = widgLayout)
        
        self.create_widgets(0, 20)
        # make initial plot
        if data: self.initializePlots(data)
        #self.create_widgets()

    def getDataSet(self, key, lowerK = 1, upperK = 1, maxJ = 20):
        '''Select data as restricted by being between lowerK and upperK and below
           maxJ'''
        restrictedData = [item for item in self.data[key] if lowerK <= int(item.K_upper) <= upperK and
                         int(item.J_upper) < maxJ]
        
        x = [int(item.J_upper) for item in restrictedData]
        y = [float(item.upperE) for item in restrictedData]
        return x, y
    
    def initializePlots(self, data):
        widget_container.clear_output()
        plt.clear()
        self.data = data
        self.keys = list(data.keys())
        self.plots = {}
        xmin, xmax = 1e20, -1e20
        ymin, ymax = 1e50, -1e50
        for i, vibration in enumerate(self.keys):
            xpoints, ypoints = self.getDataSet(vibration, 8, 11, 20)
            if xpoints:
                xmin = min(xmin, min(xpoints))
                xmax = max(xmax, max(xpoints))
                ymin = min(ymin, min(ypoints))
                ymax = max(ymax, max(ypoints))
            #print(f"for {vibration}, {len(xpoints)} points")
            self.plots[vibration] = plt.scatter(xpoints, ypoints, marker = self.markerShapes[i%6], 
                                          colors = [self.markerColors[i%7]],
                                          stroke = 'black', tooltip = self.out, unhovered_style ={'opacity':0.5},
                                          enable_hover = True)
            self.plots[vibration].on_hover(self.myToolTip)

            _ = plt.ylabel("energy (1/cm)")
            _ = plt.xlabel("J'")
            
        # attempt to extract axes information:
        axes = self.fig.axes
        x_axis, y_axis = axes
        #self.xlimits = x_axis.scale.min, x_axis.scale.max
        #self.ylimits = y_axis.scale.min, y_axis.scale.max
        
        self.xlimits = plt.xlim(xmin, xmax)
        self.ylimits = plt.ylim(ymin, ymax)
        #self.choose_minJ.value = int(xmin)
        #self.choose_maxJ.value = int(xmax)
        with self.err_out:
            print("xlimits, ylimits")
            print(self.xlimits)
            print(self.ylimits)
            print(xmin, xmax)
        self.create_widgets(xmin, xmax)
    
    def myNewToolTip(self, item, event):
        message = self.strLabel.format(item.vibration, item.L_upper, item.K_upper, item.L_upper)
        self.html_out.value = message
     
    def myToolTip(self, item, event):
        '''This creates a Tool Tip'''
        
        
        # locate the vibrational identity of selected point
        for key in self.keys:
            if self.plots[key] == item:
                break

        # find index of energy selected -- gives access to all other attributes
        idx = [float(item.upperE) for item in self.data[key]].index(event['data']['y'])
        #print("index: ", idx)
        point = self.data[key][idx]
        
        self.out.clear_output(wait=True)
        label = MakeLabel(point)
        
        with self.out:
            # creating widget here seems to help with displaying...
            display(widgets.HTML(label))
            
        self.err_out.clear_output()
        with self.err_out:
            display(widgets.HTML(label))
            

    def update_plot(self, change):
        maxJ = self.choose_maxJ.value
        for i, key in enumerate(self.keys):
            x, y = self.getDataSet(key, self.lowers[i].value, self.uppers[i].value, maxJ)
            self.plots[key].x = x
            self.plots[key].y = y
        self.ylimits.max = None
        self.ylimits.min = None
        
    def create_widgets(self, minJ, maxJ):
        # plot-limit row
        lim_layout = widgets.Layout(width = "150px")
        lim_label = widgets.Label("Change limits on J")
        self.choose_minJ = widgets.IntText(value = minJ, description = "min J", layout = lim_layout)
        self.choose_maxJ = widgets.IntText(value = maxJ, description = "max J", layout = lim_layout)
        self.btn_reset = widgets.Button(description = "reset limits", disabled = True, button_style = 'success')
        self.use_zoom = widgets.ToggleButton(value = False, description = "Zoom", button_style = 'info', icon='binoculars')
        limRow = widgets.HBox([lim_label, self.choose_minJ, self.choose_maxJ, self.btn_reset, self.use_zoom])
        # auto rows of widgets:
        int_layout = widgets.Layout(width = "125px")
        lbl_layout = widgets.Layout(width = "260px")

        self.rows = []
        self.labels = []
        self.lowers = []
        self.uppers = []
        self.shows = []
        for key in self.keys:
            self.labels.append(widgets.Label(key, layout = lbl_layout))
            self.lowers.append(widgets.IntText(value = 8, description = "min K'", layout = int_layout))
            self.uppers.append(widgets.IntText(value = 11, description = "max K'", layout = int_layout))
            self.shows.append(widgets.Checkbox(value = True, description = "Show " , layout = widgets.Layout(width = "150px")))
            self.rows.append(widgets.HBox([self.labels[-1], self.lowers[-1], self.uppers[-1], self.shows[-1]]))
            
        
        if len(self.keys) > 3:
            stack = widgets.Stack(self.rows, selected_index=0)
            self.key_drop = widgets.Dropdown(options = self.keys)
            widgets.jslink((self.key_drop, 'index'), (stack, 'selected_index'))
            selector = widgets.VBox([self.key_drop, stack])


        if self.keys:
            first_plot = self.plots[self.keys[0]]
            self.br_sel = BrushSelector(x_scale = first_plot.scales['x'], 
                               y_scale = first_plot.scales['y'], marks = [first_plot], color='lightblue')
            self.br_sel.observe(self.brush_callback, names = ['brushing'])

        for obj in [widget for row in self.rows for widget in row.children]:
            obj.observe(self.update_plot, names = 'value')
            
        self.btnSave = widgets.Button(description = "save figure", disabled = False, button_style = 'warning',
                        icon = 'download')
        self.btnSave.on_click(self.save_fig)

        self.btnShowAll = widgets.ToggleButton(value = False, description = "Show None", disabled = False, 
                                               button_style = "info", icon = 'square')
        buttonRow = widgets.HBox([self.btnSave, self.btnShowAll])
        
        self.btn_reset.on_click(self.reset_limits)
        self.use_zoom.observe(self.toggle_zoom, 'value')
        self.btnShowAll.observe(self.show_all, 'value')

        for obj in self.shows:
            obj.observe(self.toggle_vis,'value')

        for obj in [self.choose_minJ, self.choose_maxJ]:
            obj.observe(self.alterLimits, 'value')
            obj.observe(self.update_plot, 'value')
    
        #widget_container.clear_output()
        with widget_container:
            if len(self.rows) > 3:
                display( widgets.VBox([fc, self.fig, limRow, selector,  buttonRow, self.err_out]))
            else:
                display( widgets.VBox([fc, self.fig, limRow, *self.rows, buttonRow, self.err_out]))
        
    def toggle_zoom(self, change):
        '''Zoom  button callback'''
        if self.use_zoom.value:
            self.fig.interaction = self.br_sel
            self.use_zoom.description = "Show Info"
            self.use_zoom.icon = "info"
        else:
            self.fig.interaction = None
            self.use_zoom.description = "Zoom"
            self.use_zoom.icon = "binoculars"

    def toggle_vis(self, change):
        '''toggle visibility of given vibration'''
        idx = self.shows.index(change.owner)
        key = self.labels[idx].value
        self.plots[key].visible = change.new
        
    def show_all(self, change):
        '''Show none or show all of data chunks'''
        if self.btnShowAll.value:
            self.btnShowAll.description = "Show All"
            self.btnShowAll.icon = 'check-square'
        else:
            self.btnShowAll.description = "Show None"
            self.btnShowAll.icon = 'square'
        for key in self.keys:
            self.plots[key].visible = not self.btnShowAll.value
        for show in self.shows:
            show.value = not self.btnShowAll.value
            
    def alterLimits(self, change):
        self.xlimits.min = self.choose_minJ.value
        self.xlimits.max = self.choose_maxJ.value

    def reset_limits(self, stuff):
        '''Reset plot window to match chosen min/max J'''
        self.update_plot("reset")
        self.alterLimits('reset')

    def brush_callback(self, change):
        '''Callback function for zooming'''
        brushing = self.br_sel.brushing
        if not brushing:
            sel = self.br_sel.selected
            self.xlimits.min = sel[0][0]
            self.ylimits.min = sel[0][1]
            self.xlimits.max = sel[1][0]
            self.ylimits.max = sel[1][1]
            self.btn_reset.disabled = False
        self.br_sel.selected = None
    
   
    def save_fig(self, change):
        '''Saves plot in png file'''
        
        self.fig.save_png(filename = "vibration.png")

display(widget_container)  
widgets.VBox([widget_container, testing_output])
sr = SymRotViewer()


Output()

In [6]:
# extract datafile
class CallBackClass:
    def __init__(self):
        self.return_value = 'Empty'
        self.data = {}
        self.blockStarts = []
        
    def callback_func(self, chooser):
        filename = chooser.selected
        self.return_value = open(filename, 'r').readlines()
        # new section
        flines = self.return_value
        self.blockStarts = blockStarts = []
        for n, line in enumerate(flines):
            if line.strip().startswith("UPPER VIB"):
                blockStarts.append(n-1)
        blockStarts.append(len(flines)-7)
        
        data = {}
        for i,n in enumerate(blockStarts[:-1]):
            block = flines[n+5: blockStarts[i+1]-3]
            vibration = flines[n].strip()
            data[vibration] = []
            for line in block:
                data[vibration].append(dataline(vibration, *line.split()))
        self.data = data
        sr.initializePlots(data)
        
cb = CallBackClass()
fc.register_callback(cb.callback_func)